# Analyze for the books store

In [4]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'': '', 
 'pwd': '', 
 'host': '',
 'port': , 
 'db': ''} #connection was hidden by artemtu
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Подключение к ДБ и знакомство с данными
<a id="start"></a>

In [5]:
def sql_result(query):
  return pd.io.sql.read_sql(query, con = engine)


In [6]:
query = \
'''

SELECT * 
FROM books
LIMIT 3

'''
sql_result(query)



,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


In [7]:
query = \
'''

SELECT * 
FROM authors
LIMIT 5

'''
sql_result(query)


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
query = \
'''

SELECT * 
FROM publishers
LIMIT 3

'''
sql_result(query)


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


In [9]:
query = \
'''

SELECT * 
FROM ratings
LIMIT 3

'''
sql_result(query)


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


In [10]:
query = \
'''

SELECT * 
FROM reviews
LIMIT 3

'''
sql_result(query)



,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


Connection to the database is successful

## How many books were published after the 1 of January 2000
<a id="step1"></a>

In [11]:
query = \
'''

SELECT COUNT (book_id) as book_cnt
FROM books
WHERE publication_date > '2000-01-01'

'''
sql_result(query)

,book_cnt
0,819


819 books

## For every book count reviews quantity and average grade
<a id="step2"></a>

In [12]:
query = \
'''

SELECT
  books.title,
  COUNT (distinct review_id) as review_cnt,
  AVG(rating) AS avg_rating
FROM
  books
LEFT JOIN
  reviews ON reviews.book_id = books.book_id
LEFT JOIN
  ratings ON ratings.book_id = books.book_id
GROUP BY
  books.title
ORDER BY 
    COUNT(reviews.review_id) DESC

'''
sql_result(query)


,title,review_cnt,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
995,Essential Tales and Poems,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,The Natural Way to Draw,0,3.000000


## Find a publishing house which published books with more than 50 pages
<a id="step3"></a>

In [13]:
query = \
'''

SELECT
  publisher,
  COUNT (book_id) as book_cnt
FROM
  books
LEFT JOIN
  publishers ON publishers.publisher_id=books.publisher_id
WHERE
  num_pages>50
GROUP BY
  publisher
ORDER BY
  book_cnt DESC

'''
sql_result(query)

,publisher,book_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


## Find authors which a high grade on the books. We should find books with more than 50 pages

<a id="step4"></a>

In [14]:
query = \
'''
SELECT
  author,
  AVG(sub.avg_rating) AS high_avg_rating
FROM
(SELECT
  author,
  books.book_id,
  AVG(rating) AS avg_rating
FROM 
  authors
LEFT JOIN books ON books.author_id = authors.author_id
LEFT JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
  author,
  books.book_id
HAVING
  COUNT(rating) > 50) AS sub
GROUP BY
  author
ORDER BY
  high_avg_rating DESC
'''

sql_result(query)

,author,high_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


## Find average user reviews which sent more than 50 reviews
<a id="step5"></a>

In [15]:
query = \
'''
SELECT
  AVG(review_cnt) as avg_review_cnt
FROM
(SELECT
  COUNT(review_id) as review_cnt
FROM
  reviews
WHERE username IN
(SELECT
  username
FROM
  ratings
GROUP BY
  username
HAVING
  COUNT(ratings) > 50)
GROUP BY
  username) as sub_review_cnt 

'''
sql_result(query)

,avg_review_cnt
0,24.333333


Average user reviews which sent more than 50 reviews - 24